## Optimized_Parser_approach CICIDS


This approach comprises of extracting pcap file features utilizing own developed pcap parser. Then utilizing it for comparison and labelling. Prominent steps for this approach are:
1. Extract Pcap file features into csv using developed parser. 
2. Perform labelling by comparing pre-processed UNSW CSV file and pcap parsed file (in .csv format).This approach compares both file using 5 distinct features which are :

> a)  Source IP Address(*srcip*) 
>
> b)  Destination IP Address(*dstip*)
>
> c)  Destination Port Number(*dsport*)
>
> d)  Source Port Numbe(*sport*)
>
> e)  Transaction Protocol(*protocol_m*)

3. Moreover, time duration is utilized to cross validate the attacks. The time duration is available at the official site of the dataset.
            



In [2]:
import sys
sys.path.append("C:\\Users\\yasir.ali\\Project\\Payload-Byte")

In [3]:
import hashlib as hl
import csv
from datetime import datetime
from os import close
import numpy as np
import logging
import argparse
import sys
from datetime import datetime
import socket
import math
import pickle
import random
import pandas as pd
from functions.Functions_wireshark import *
from functions.Optimized_Parser_Labelling import *
log = logging.getLogger()
log.setLevel(logging.DEBUG)

In [84]:
## Checking ambiguity in the given time
df_p['Check']=df_p['Timestamp'].apply(lambda x: 100 if x.count(':') == 1 else 200)
df_p.Check.value_counts()

100    2298227
200     529450
Name: Check, dtype: int64

## Reading pre-processed CSV File of CICIDS


In [4]:
df_p=pd.read_csv('F:/Project_Data/NIDS/CSV-Files-CICIDS-2017/Preprocessed-Data/CICIDS.csv')
df_p=df_p[['Timestamp','Source IP','Destination IP','Destination Port','Source Port','Protocol','Label']]
df_p.rename(columns={'Timestamp': 'stime', 'Source IP': 'srcip', 'Destination IP': 'dstip', 'Destination Port': 'dsport', 'Source Port': 'sport', 'Label': 'label','Protocol': 'protocol_m'}, inplace=True)

# Converting time into epoch format along with the mitigation of AM/PM issue.
df_p['stime']=df_p['stime'].apply(lambda x: (datetime.strptime(x, '%d/%m/%Y %H:%M')) if x.count(':') == 1 else  (datetime.strptime(x, '%d/%m/%Y %H:%M:%S')))
df_p['stime']=df_p['stime'].apply(lambda x: int((datetime(x.year,x.month,x.day,(x.hour+12),x.minute,x.second)).timestamp()) if (x.hour>=1)&(x.hour<=7) else int((datetime(x.year,x.month,x.day,x.hour,x.minute,x.second)).timestamp()) )
df_p=df_p.sort_values(by='stime')

In [6]:
df_p['protocol_m'] = df_p['protocol_m'].astype(str)
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('6.0', 'tcp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('17.0', 'udp'))
df_p['protocol_m']=df_p['protocol_m'].apply(lambda x: x.replace('0.0', 'other'))

In [7]:
df_p.label.value_counts()

BENIGN                        2271122
DoS Hulk                       230123
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack – Brute Force         1507
Web Attack – XSS                  652
Infiltration                       36
Web Attack – Sql Injection         21
Heartbleed                         11
Name: label, dtype: int64

## Parsing PCAP files into CSV

In [1]:
pcap='F:/Project_Data/NIDS/Pcap-Files-CICIDS-2017/'
pcap_file=[]
days=['Monday','Tuesday','Wednesday','Thursday','Friday']
for x in days:
    pcap_file.append(pcap+x+"-WorkingHours"+".pcap")
    
    
out_file='F:/Project_Data/NIDS/CICIDS_result/pcap_file_csv_parser/'

In [ ]:
pcap_parser(pcap_file,out_file,1)

## Labeling & Combining of Parsed PCAP files

In [8]:
df=pd.read_csv("F:/Project_Data/NIDS/CICIDS_result/pcap_file_csv_parser/pcap_csv_1.csv",index_col=0)
df=df.sort_values(by='stime')

In [9]:
# df=df.sort_values(by='stime')
stime=df.stime[0]
ltime=int(df.stime.tail(1))
print("Start Value:",df.stime[0])
print("End Value:", int(df.stime.tail(1)))

Start Value: 1499082959
End Value: 1499112094


In [358]:
df.drop(columns=['frame_num','stime','ltime','protocol_s'],inplace=True)

In [10]:
aa=df_p[(df_p['stime']>=stime) & (df_p['stime']<=ltime)]

In [12]:
df_p.shape

(2827677, 7)

In [12]:
df.protocol_m.value_counts()

tcp     10710230
udp       759022
icmp         444
igmp          40
Name: protocol_m, dtype: int64

In [362]:
aa.protocol_m.value_counts()

tcp      285044
udp      142291
other       231
Name: protocol_m, dtype: int64

### `Labelling & Analyzing`


In [395]:
combine=df.merge(aa, left_on=['srcip','dstip','dsport','sport','protocol_m'], right_on=['srcip','dstip','dsport','sport','protocol_m'])
combine

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime,label
0,192.168.10.50,56108,192.168.10.3,3268,tcp,64,455,0000018f6082018b06092a864886f71201020202011100...,0.000000,1499435940,BENIGN
1,192.168.10.50,56108,192.168.10.3,3268,tcp,64,455,0000018f6082018b06092a864886f71201020202011100...,0.000000,1499436060,BENIGN
2,192.168.10.50,56108,192.168.10.3,3268,tcp,64,455,0000018f6082018b06092a864886f71201020202011100...,0.000000,1499436240,BENIGN
3,192.168.10.50,56108,192.168.10.3,3268,tcp,64,455,0000018f6082018b06092a864886f71201020202011100...,0.000000,1499436360,BENIGN
4,192.168.10.50,56108,192.168.10.3,3268,tcp,64,455,0000018f6082018b06092a864886f71201020202011100...,0.000000,1499436480,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...
19740070,192.168.10.15,59585,205.174.165.73,8080,tcp,128,52,NaN,0.130817,1499462160,BENIGN
19740071,192.168.10.15,59585,205.174.165.73,8080,tcp,128,48,NaN,0.001495,1499462160,BENIGN
19740072,192.168.10.8,9807,205.174.165.73,8080,tcp,128,52,NaN,0.014476,1499462160,BENIGN
19740073,192.168.10.8,9807,205.174.165.73,8080,tcp,128,52,NaN,0.037433,1499462160,BENIGN


In [396]:
combine.drop_duplicates(inplace=True)

In [397]:
combine.shape

(8830316, 11)

In [ ]:
combine.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/labelled_pcap_csv_5.csv",index=False)

## Cross Validation of Time Frame of Attacks

## Friday

In [400]:
##  DDoS LOIT (15:56 – 16:16)
combine.drop(combine[(combine.stime>=1499453700 )&(combine.stime<=1499459400 )&(combine.label=='BENIGN')].index,inplace=True)

In [401]:
## Port Scan
combine.drop(combine[(combine.stime>=1499453700 )&(combine.stime<=1499459400 )&(combine.label=='BENIGN')].index,inplace=True)

In [399]:
## Botnet ARES (10:02 a.m. – 11:02 a.m.)
combine.drop(combine[(combine.stime>=1499439720 )&(combine.stime<=1499443320 )&(combine.label=='BENIGN')].index,inplace=True)

In [402]:
combine.shape

(5848193, 11)

## Thursday

In [349]:
## Infiltration – Dropbox download (15:04 – 15:45 p.m.)
combine.drop(combine[(combine.stime>=1499371200)&(combine.stime<=1499373900)&(combine.label=='BENIGN')].index,inplace=True)

In [350]:
## Infiltration – Dropbox download and (14:33 -14:35)
combine.drop(combine[(combine.stime>=1499369400)&(combine.stime<=1499369700)&(combine.label=='BENIGN')].index,inplace=True)

In [351]:
## Web Attack – Sql Injection (10:40 – 10:42 a.m.)
combine.drop(combine[(combine.stime>=1499355600)&(combine.stime<=1499355720)&(combine.label=='BENIGN')].index,inplace=True)

In [352]:
## Web Attack – XSS (10:15 – 10:35 a.m.)
combine.drop(combine[(combine.stime>=1499354100)&(combine.stime<=1499355300)&(combine.label=='BENIGN')].index,inplace=True)

In [353]:
## Web Attack – Brute Force (9:20 – 10 a.m.)
combine.drop(combine[(combine.stime>=1499350800)&(combine.stime<=1499353200)&(combine.label=='BENIGN')].index,inplace=True)

In [354]:
combine.shape

(4071767, 11)

## Wednesday

In [303]:
## Heartbleed Port 444 (15:12 - 15:32)
combine.drop(combine[(combine.stime>=1499285520)&(combine.stime<=1499286720)&(combine.label=='BENIGN')].index,inplace=True)

In [302]:
## DoS GoldenEye (11:10 – 11:23 a.m.)
combine.drop(combine[(combine.stime>=1499271000)&(combine.stime<=1499271780)&(combine.label=='BENIGN')].index,inplace=True)

In [301]:
## DoS Hulk (10:43 – 11 a.m.)
combine.drop(combine[(combine.stime>=1499269380)&(combine.stime<=1499270400)&(combine.label=='BENIGN')].index,inplace=True)

In [300]:
##DoS Slowhttptest (10:14 – 10:35 a.m.)
combine.drop(combine[(combine.stime>=1499267640)&(combine.stime<=1499268900)&(combine.label=='BENIGN')].index,inplace=True)

In [299]:
## DoS slowloris (9:47 – 10:10 a.m.)
combine.drop(combine[(combine.stime>=1499266020)&(combine.stime<=1499267400)&(combine.label=='BENIGN')].index,inplace=True)

In [306]:
combine.shape

(16436931, 11)

## Tuesday

In [212]:
## FTP-Patator (9:20 – 10:20 a.m.)
combine.drop(combine[(combine.stime>=1499177940)&(combine.stime<=1499181660)&(combine.label=='BENIGN')].index,inplace=True)

In [220]:
1499194800.0
##SSH-Patator (14:00 – 15:00 p.m.)
combine.drop(combine[(combine.stime>=1499194800)&(combine.stime<=1499198400)&(combine.label=='BENIGN')].index,inplace=True)

## Reading and Combining All Labelled Files

In [2]:
def combine_CICIDS(in_file_path,out_path):
    combine=pd.DataFrame(columns=['srcip', 'sport', 'dstip', 'dsport', 'protocol_m',
     'sttl', 'total_len', 'payload', 't_delta', 'stime','label'])
    for files in in_file_path:
        df=pd.read_csv(files)
        combine=combine.append(df, ignore_index=True)
        print(combine.shape)
    csv_out=out_path+"combined_labelled_pcap_csv.csv"
    logging.info("Exporting_combined_csv_file....")
    combine.to_csv(csv_out,index=False)
    return combine

In [3]:
label_csv="F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/"
in_file=[]
for x in range(1,6): ## Starting & Ending values for the files
    in_file.append(label_csv+"labelled_pcap_csv_"+str(x)+".csv")
out_path="F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/"

In [4]:
df_payload=combine_CICIDS(in_file,out_path)

(5633567, 11)
(9150136, 12)
(25587067, 12)
(29658834, 12)


INFO:root:Exporting_combined_csv_file....


(35507027, 12)


In [7]:
df_payload.shape

(35507027, 12)

In [6]:
df_payload.label.value_counts()

BENIGN                        23154473
DoS Hulk                       9211652
DDoS                           1495765
DoS GoldenEye                   489471
DoS slowloris                   375252
DoS Slowhttptest                312365
PortScan                        166146
Infiltration                    115097
SSH-Patator                      66573
FTP-Patator                      50141
Web Attack – Brute Force         27948
Heartbleed                       23320
Bot                              10422
Web Attack – XSS                  8335
Web Attack – Sql Injection          67
Name: label, dtype: int64

In [11]:
df_payload.drop(df_payload[df_payload.payload.isnull()].index,inplace=True)

In [116]:
df_payload.shape

(19367138, 13)

In [100]:
x=df_payload['payload']
new=[]
for p in range(len(x)):
    try:
        o=(int((x.iloc[p]), 16))
        if o>0:
            new.append(1)
        else:
            new.append(0)
    except Exception as e:
        print(p)
        new.append('Big')

In [115]:
df_payload['payload_int']=new

In [117]:
df_payload.drop(df_payload[df_payload.payload_int==0].index,inplace=True)

In [119]:
## After Removing zero payload
df_payload.label.value_counts()

BENIGN                        16073175
DoS Hulk                       2495632
DDoS                            241405
DoS GoldenEye                   128122
DoS slowloris                   121097
Infiltration                    115007
DoS Slowhttptest                 80542
SSH-Patator                      48165
FTP-Patator                      31843
Heartbleed                       13486
Web Attack – Brute Force         11754
Web Attack – XSS                  3341
Bot                               2543
PortScan                           830
Web Attack – Sql Injection          12
Name: label, dtype: int64

In [ ]:
df_payload.pop('payload_int')

In [133]:
df_payload.sttl=df_payload.sttl.astype('int32')
df_payload.dsport=df_payload.dsport.astype('int32')
df_payload.sport=df_payload.sport.astype('int32')
df_payload.total_len=df_payload.total_len.astype('int32')

In [136]:
df_payload.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/combined_labelled_cleaned_sorted_pcap_csv.csv",index=False)

In [140]:
df_payload.iloc[:,:-1]

,srcip,sport,dstip,dsport,protocol_m,sttl,total_len,payload,t_delta,stime
8,192.168.10.9,1034,192.168.10.3,88,tcp,128,1437,000005716c82056d30820569a103020105a20302010ca3...,0.000002,1.499090e+09
12,192.168.10.9,1034,192.168.10.3,88,tcp,128,1437,000005716c82056d30820569a103020105a20302010ca3...,0.000046,1.499090e+09
15,192.168.10.3,88,192.168.10.9,1034,tcp,128,1450,0000057e6d82057a30820576a003020105a10302010da3...,0.000049,1.499090e+09
16,192.168.10.3,88,192.168.10.9,1034,tcp,128,1450,0000057e6d82057a30820576a003020105a10302010da3...,0.000284,1.499090e+09
20,192.168.10.9,1030,192.168.10.3,445,tcp,128,158,0900000048002a005c005c00440043002e005400650073...,0.000422,1.499090e+09
...,...,...,...,...,...,...,...,...,...,...
35506993,192.168.10.16,43619,192.168.10.3,53,udp,64,60,ecb6010000010000000000000377777706706172616c79...,0.000333,1.499462e+09
35506994,192.168.10.16,43619,192.168.10.3,53,udp,64,60,ecb6010000010000000000000377777706706172616c79...,0.000004,1.499462e+09
35506995,192.168.10.16,10009,192.168.10.3,53,udp,64,60,4f5b010000010000000000000377777706706172616c79...,0.000001,1.499462e+09
35506996,192.168.10.16,38920,192.168.10.3,53,udp,64,56,3663010000010000000000000264730461647667026a70...,0.000001,1.499462e+09


## Reducing Number of Instances for Dataset

In [137]:
from imblearn.under_sampling import RandomUnderSampler 

In [143]:
dict1={ 'BENIGN': 362108,
'DoS Hulk':          250000,
'DDoS'  :         241405,
'DoS GoldenEye':     128122,
'DoS slowloris':    121097,
'Infiltration'        :115007,
'DoS Slowhttptest'         :  80542,
'SSH-Patator':          48165,
'FTP-Patator'            :   31843,
'Heartbleed'              :  13486,
'Web Attack – Brute Force'            :   11754,
'Web Attack – XSS'              :  3341,
'Bot'            :   2543,
'PortScan'              :  830,
'Web Attack – Sql Injection': 12
}

In [144]:
rus = RandomUnderSampler(random_state=42,sampling_strategy=dict1)

In [145]:
X_res, y_res = rus.fit_resample(df_payload.iloc[:,:-1], df_payload.iloc[:,-1])

In [149]:
X_res['label']=y_res

In [150]:
X_res.to_csv("F:/Project_Data/NIDS/CICIDS_result/Labelled_pcap_file/Combined/combined_labelled_cleaned_undersampled_pcap_csv.csv",index=False)

## Transformation into Byte-wise features

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(X_res['protocol_m'])
X_res['protocol']=label

X_res.drop("protocol_m", axis=1, inplace=True)
X_res.drop("protocol_s", axis=1, inplace=True)
X_res.drop("frame_num", axis=1, inplace=True)

X_res['attack_cat']=y_res

In [ ]:
X_tr,Ytrain =payload_to_bytes(X_res,1500)

In [ ]:
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,6])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,7])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,5])))
X_tr = np.column_stack((X_tr,np.array(X_res.iloc[:,9])))

In [ ]:
name=[]
for x in range(1,1501):
    name.append("payload_byte_"+str(x))
name.append("ttl")
name.append("total_len")
name.append("protocol")
name.append("t_delta")

In [ ]:
df = pd.DataFrame(X_tr, columns=name)
df['label']=Ytrain

In [ ]:
df.to_csv("F:/Converted_data_CICIDS.csv",index=False)

## Extra Code/ Conversion Epoch

In [ ]:
df_payload['stime'].fillna(df_payload['stime_y'],inplace=True)
df_payload.pop('stime_y')

### Transformation into epoch Format

In [377]:
x='07/07/2017 16:16:00'
m2 = datetime.strptime(x, '%d/%m/%Y %H:%M:%S').timestamp()
print(m2)

1499462160.0


In [ ]:
ts_epoch =1499457761
ts = datetime.fromtimestamp(ts_epoch).strftime('%d/%m/%Y %H:%M:%S')
ts